# 明日方舟抽卡 六星概率分析
使用 普通方法/Markov Chain方法/Monte Carlo方法 分析明日方舟抽卡获得六星的综合概率

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb

## 1. 普通方法
借用[十连白光分析](http://localhost:8888/notebooks/ten-pull_events_analyses.ipynb)中的思路与代码

考虑一个无限次抽卡的序列，每相邻两次六星之间的抽数间隔是一个随机变量，记作$N_{int}$。六星卡在序列中占的比例即为综合概率$p_6'$，有
$$
    p_6' = \frac{1}{\mathbf{E}[N_{int}]+1}
$$
该抽数期望在玩家中已基本达成共识($\mathbf{E}[N_{int}]+1=34.59$)，为保证结论严谨，仍将计算过程写出。

将“连续n抽没有六星的前提下，下一抽抽到六星的概率”记为$\mathbf{P}[6 | N_{int}=n]$。
$$
    \mathbf{P}[6 \mid N_{int}=n] = 0.02, \quad\quad\quad\quad n=0,1,\ldots,49\quad \\
    \mathbf{P}[6 \mid N_{int}=n] = 0.02 (n-48), \quad n=50,51,\ldots,98
$$

将“连续n抽没有六星，且第n+1抽抽到六星的概率”记为$\mathbf{P}[N_{int}=n]$。
$$
    \mathbf{P}[N_{int}=n] = \prod_{k=0}^{n-1} (1-\mathbf{P}[6 \mid N_{no6}=k]) 
        \times \mathbf{P}[6 \mid N_{int}=n]
$$

最后可求得
$$
    p_6' = 0.0289
$$

以下为有关代码

In [2]:
# some constants
p6 = 0.02
n_nor = 50 # pulls with normal probability 
n_abnor = int((1-p6)/0.02) # pulls with increased probability

# after n pulls without 6, the probability of pull 6. n in 0-98
p6_list = [p6 for n in range(0,n_nor)] + [0.02*(n-n_nor+2) for n in range(n_nor,n_nor+n_abnor)] 
p6_list = np.array(p6_list)

# after probability of n pulls without 6 and pull six. n in 0-98
pint_list = [np.prod(1-p6_list[0:n]) * p6_list[n] for n in range(0,99)]
pint_list = np.array(pint_list)

# the comprehensive probability of pulling 6
N_exp = np.sum(np.arange(99)*pint_list)+1
p6_prime = 1/N_exp

print("抽出六星干员需要的抽数期望为：{:.4g}".format(N_exp))
print("不触发保底时，抽出六星干员的概率为：{:.4g}".format(p6))
print("考虑保底，抽出六星干员的综合概率为：{:.4g}".format(p6_prime))

抽出六星干员需要的抽数期望为：34.59
不触发保底时，抽出六星干员的概率为：0.02
考虑保底，抽出六星干员的综合概率为：0.02891


## 2. Markov Chain方法
将抽卡看作一个Markov过程，定义“与上次抽到六星间隔$n$抽”为状态$n$，$n=0,1,2,\ldots,98$。处于状态$k$时，一次抽卡产生一次状态转移：抽到六星，就转移至状态0；没抽到六星，就转移到状态$k+1$。

状态转移矩阵为$P$，定义如下(保险起见，用code cell写latex，markdown cell在GitHub上有时会显示错误)

In [3]:
%%latex
$$
    P = \begin{bmatrix}
        P_1 & \begin{bmatrix}
            P_2 \\ \mathbf{0}
        \end{bmatrix}
    \end{bmatrix}
    \\ \\
$$
其中 $P_1$ 是一个99x1的列向量，前50个元素均为0.02，第51-99个元素为0.04,0.06,...1。
$$
    P_1 = [0.02, 0.02, \ldots, 0.02, 0.04, 0.06, \ldots, 0.98, 1]^T
    \\ \\
$$

$P_2$ 是一个98x98的对角矩阵，对角线上前50个元素为0.98，第51-98个元素为0.96,0.94,...0.02。这样，恰保证$P$每行元素之和均为1。
$$
    P_2 = diag([0.98, 0.98, \ldots, 0.98, 0.96, 0.94, \ldots, 0.04, 0.02])
    \\ \\
$$
最终，得到的 $P$ 是一个99x99的矩阵，形式如下
$$
    P = \begin{bmatrix}
        0.02 & 0.98 & 0 & \dots &&&& \dots & 0 \\
        0.02 & 0 & 0.98 & \dots &&&& \dots & 0 \\
        \vdots &&& \ddots &&&&& \vdots \\
        0.02 & 0 & \dots && 0.98 &&& \dots \\
        0.04 & 0 & \dots &&& 0.96 && \dots \\
        0.06 & 0 & \dots &&&& 0.94 & \dots \\
        \vdots &&&&&&& \ddots \\
        0.98 & 0 & \dots &&&&& \dots & 0.02 \\
        1 & 0 & \dots &&&&& \dots & 0 \\
    \end{bmatrix}
$$

<IPython.core.display.Latex object>

可直接求解该Markov Chain的稳定分布$\pi$来算出六星出率。出率是如何与稳定分布$\pi$联系起来的呢？

因为“抽出六星” = 进入“与上次抽到六星间隔0抽”的状态 = 转移到状态0。而稳定分布是转移次数趋于无穷大时的分布，稳定分布处于状态0的概率，就是无穷多抽后，每一抽的六星出率。

$pi$满足条件：
$$
    \pi P = \pi \\
    \pi \mathbf{i} = 1
$$
其中$\mathbf{i}$是值均为1的列向量。令
$$
    A = \begin{bmatrix}
        \begin{bmatrix}
            (P^T - I)
        \end{bmatrix} \\
        \mathbf{i}^T
    \end{bmatrix} \\
    \mathbf{b} = [0,0,\ldots,0,1]^T
$$
有
$$
    A \pi^T = \mathbf{b}
$$
实际求解时，需解(据说是依据Rouché–Capelli theorem，我觉得用SVD也可以解释？)
$$
    (A^T A) \pi^T = A^T \mathbf{b}
$$

In [4]:
P1 = np.expand_dims(p6_list,1)
P2_prime = np.diag(1-p6_list)
P = np.concatenate([P1,P2_prime],1)[:,:-1]
I = np.diag(np.ones_like(p6_list))
A = np.pad(np.transpose(P)-I,
    ((0,1),(0,0)), mode='constant', constant_values=1)
b = np.zeros_like(A[:,0])
b[-1] = 1
print("状态转移矩阵为：")
print(P)

状态转移矩阵为：
[[0.02 0.98 0.   ... 0.   0.   0.  ]
 [0.02 0.   0.98 ... 0.   0.   0.  ]
 [0.02 0.   0.   ... 0.   0.   0.  ]
 ...
 [0.96 0.   0.   ... 0.   0.04 0.  ]
 [0.98 0.   0.   ... 0.   0.   0.02]
 [1.   0.   0.   ... 0.   0.   0.  ]]


In [5]:
pi_vec = np.linalg.solve(np.transpose(A)@A, np.transpose(A)@b)

assert np.abs(pi_vec[0]-p6_prime) < 1e-8, "Markov方法与普通方法结果不同？？！"
print("六星出率为：{:.4g}，与普通方法结果一致".format(pi_vec[0]))

六星出率为：0.02891，与普通方法结果一致


## 3. Monte Carlo方法
尚在编程中...